In [0]:
import os
import pathlib
import random
import tensorflow as tf
import tensorflow_hub as tfhub
#tf.enable_eager_execution()

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_steps = 500 # Modelo Train
summary_steps = 100 # Modelo Train
steps_per_epoch = 100 # Modelo Train
log_step = 10 # Modelo Train
train_batch_size = 5 # Funciones de input train y eval
BATCH_SIZE = 5 # dataset buffer y batch
eval_delay_secs = 10 # Modelo Eval
throttle_secs = 30 # Modelo Eval

In [16]:
tf.logging.info('TF Version {}'.format(tf.__version__))
tf.logging.info('GPU Available {}'.format(tf.test.is_gpu_available()))
if 'TF_CONFIG' in os.environ:
    tf.logging.info('TF_CONFIG: {}'.format(os.environ["TF_CONFIG"]))

INFO:tensorflow:TF Version 1.13.1


I0429 06:40:13.016022 140209631766400 <ipython-input-16-33f139de6631>:1] TF Version 1.13.1


INFO:tensorflow:GPU Available True


I0429 06:40:13.031624 140209631766400 <ipython-input-16-33f139de6631>:2] GPU Available True


**Rutas**

In [0]:
# Modelo general
path_model = '/content/model/'
os.makedirs(os.path.join(path_model), exist_ok=True)

# Modelo entrenado
path_trained = os.path.join(path_model, 'trained')

# Modelo importado
TFHUB_CACHE_DIR = os.path.join(path_model, 'TFHub')
os.environ['TFHUB_CACHE_DIR'] = TFHUB_CACHE_DIR

# Data importada
path_data = os.path.join(path_model, 'dogscats')
url_data = 'http://files.fast.ai/data/dogscats.zip'

# Modelo importado
**Importacion**

In [0]:
if not os.path.isdir(os.path.join(path_model, 'TFHub')):
  classifier_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2" #@param {type:"string"}
  tfh_module = tfhub.Module(classifier_url)

**Variables derivadas**

In [0]:
#IMAGE_SIZE = tfhub.get_expected_image_size(tfh_module)

import shutil
shutil.rmtree('/content/model/trained', ignore_errors=False, onerror=None)

# Tensorboard

In [0]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip
#
#LOG_DIR = path_trained
#get_ipython().system_raw(
#    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
#    .format(LOG_DIR))
#
#get_ipython().system_raw('./ngrok http 6006 &')
#
#! curl -s http://localhost:4040/api/tunnels | python3 -c \
#    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Datos


## Adquisicion

In [0]:
if not os.path.isdir(path_data):
  tf.keras.utils.get_file(fname = os.path.basename(url_data), origin = url_data, 
                          cache_subdir=path_model, extract=True)

**Directorios de datos**

In [0]:
train_path = os.path.join(path_data, 'train')
eval_path = os.path.join(path_data, 'valid')
pred_path = os.path.join(path_data, 'sample', 'test/')

---
For this output, you can connect additional layers according to the problem you want to solve. For example, if the problem is to classify images into 10 categories, you may want to continue fully connected-layer with 10 nodes, and apply the softmax function to have the probabilities.


logits = tf.layers.dense(inputs=outputs, units=10)
predictions = {
    "classes": tf.argmax(input=logits, axis=1),
    "probabilities": tf.nn.softmax(logits)
}

---
train_input_fn = tf.data.Dataset.list_files(train_files)

---
label = tf.string_split(source = [train_files], delimiter = '/').values[-2]

---
from IPython.core.debugger import Tracer; Tracer()() 
n, c, q

# Dataset

In [0]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3) #<-------
  image = tf.image.resize(image, [224, 224]) #<-------
  image /= 255.0  # normalize to [0,1] range #<-------
  return image

In [0]:
def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)

In [0]:
def read_dataset(filename, tfr_file, mode, batch_size):  

  data_root = pathlib.Path(filename)#<------- filename
  all_image_paths = list(data_root.glob('*/*'))
  all_image_paths = [str(path) for path in all_image_paths]
  random.shuffle(all_image_paths)

  if not os.path.isfile(tfr_file):    #<------- tfr_file
    paths_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
    image_ds = paths_ds.map(load_and_preprocess_image)
    ds = image_ds.map(tf.serialize_tensor) 
    tfrec = tf.data.experimental.TFRecordWriter(tfr_file)#<------- tfr_file
    tfrec.write(ds)

  #Toma la ruta de las imagenes, la decodifica, crea el tensor de atributos
  ds = tf.data.TFRecordDataset(tfr_file)
  def parse(x):
    result = tf.parse_tensor(x, out_type=tf.float32)
    result = tf.reshape(result, [224, 224, 3]) #<-------
    return result
  ds = ds.map(parse, num_parallel_calls=AUTOTUNE)
  
  #Como mejorar estas lineas
  label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
  label_to_index = dict((name, index) for index,name in enumerate(label_names))
  all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
  paths_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
  image_ds = paths_ds.map(load_and_preprocess_image)
  label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
  
  ds = tf.data.Dataset.zip((image_ds, label_ds)) #<-------
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    num_epochs = None #Loop indefinidamente
    ds = ds.shuffle(buffer_size = batch_size) # video 7:10 https://www.youtube.com/watch?v=uIcqeP7MFH0
    ds = ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=batch_size*10)) #image_count
  else:
    num_epochs = 1 #end-of-input after this
  
  ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE) # video 7:48
  return ds

In [0]:
def train_input_fn():
  return read_dataset(filename = train_path,#<-------
                      tfr_file = '/content/model/train.tfrec',
                      batch_size = train_batch_size, 
                      mode = tf.estimator.ModeKeys.TRAIN)

def eval_input_fn():
  return read_dataset(filename = eval_path,#<-------
                      tfr_file = '/content/model/eval.tfrec',
                      batch_size = train_batch_size, 
                      mode = tf.estimator.ModeKeys.EVAL)

def pred_input_fn():
  return read_dataset(filename = pred_path,#<------- 
                      tfr_file = '/content/model/pred.tfrec',
                      batch_size = train_batch_size, 
                      mode = tf.estimator.ModeKeys.EVAL)

def serving_input_fn():
  json_feature_placeholders = {'images' : tf.placeholder(tf.float32, shape=(None, 224, 224, 3))} #<--- puede ser shape(None)
  features = json_feature_placeholders
  return tf.estimator.export.ServingInputReceiver(features, json_feature_placeholders)

# Modelo
**Configuracion del modelo importado**

In [0]:
def model_fn(features, labels, mode, params):
  #module = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_050_192/feature_vector/2")
  #module = hub.Module("https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/1")
  tfh_module=tfhub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2")
  transformed_features = tfh_module(features)
  logits = tf.layers.dense(transformed_features, 2) #<-------
  probabilities = tf.nn.softmax(logits)
  
  if (mode != tf.estimator.ModeKeys.PREDICT):
    one_hot_labels = tf.one_hot(labels, 2) #<-------
    loss = tf.losses.softmax_cross_entropy(one_hot_labels, logits)
    optimizer = tf.train.AdamOptimizer() 
    train_op = tf.contrib.training.create_train_op(loss, optimizer)
    accuracy = tf.metrics.accuracy(labels, tf.argmax(probabilities, axis=-1))
    metrics = {'acc':accuracy}
  else:
    # None of these can be computed in prediction mode because labels are not available
    loss = optimizer = train_op = metrics = None
  
  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=train_op,
      # Nice: in estimator, you can return computed results that are not part of model training (here: class)
      predictions={'proba': probabilities, 'class': tf.argmax(probabilities, axis=-1)},
      eval_metric_ops=metrics
  )

# Entrenamiento

In [0]:
def train_and_evaluate(): #args
  # Estimador del modelo a usar
  run_config = tf.estimator.RunConfig(
    model_dir = path_trained, #args['output_dir'],  Ouput directory for checkpoint
    save_summary_steps = summary_steps, 
    save_checkpoints_steps = steps_per_epoch, #, save_checkpoints_step = 100
    log_step_count_steps=log_step)

  # Define los aspectos del modelo # ojo aqui esta el transfer learning
  model = tf.estimator.Estimator(
    model_fn = model_fn,
    config = run_config)

  # Define los aspectos del entrenamiento y la entrada de datos
  train_spec = tf.estimator.TrainSpec(
    input_fn = train_input_fn, 
    max_steps = train_steps) #args['train_steps']

  # Define los aspectos del uso en produccion con ML Engine
  export_latest = tf.estimator.LatestExporter(
    'exporter', #folder to export
    serving_input_receiver_fn = serving_input_fn)

  # Define los aspectos de la evaluacion, cada cuanto se graba para tensorboard y la entrada de datos
  eval_spec = tf.estimator.EvalSpec(
    input_fn = eval_input_fn,
    steps = None,
    start_delay_secs = eval_delay_secs, # args['eval_delay_secs'] start evaluating after N seconds
    throttle_secs = throttle_secs, #, args['throttle_secs'] evaluate every N seconds
    exporters = export_latest)

  # Ejecuta el modelo
  out = tf.estimator.train_and_evaluate(model, train_spec, eval_spec)
  return out

# Prediccion

In [29]:
import shutil
if os.path.isdir('/content/model/trained'):
  shutil.rmtree('/content/model/trained', ignore_errors=False, onerror=None)

trained_model = train_and_evaluate()
print(trained_model)

INFO:tensorflow:Using config: {'_model_dir': '/content/model/trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f84a761bb00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0429 06:40:13.209277 140209631766400 estimator.py:201] Using config: {'_model_dir': '/content/model/trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f84a761bb00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


W0429 06:40:13.212150 140209631766400 estimator.py:1924] Estimator's model_fn (<function model_fn at 0x7f84a7617488>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Not using Distribute Coordinator.


I0429 06:40:13.214707 140209631766400 estimator_training.py:185] Not using Distribute Coordinator.


...

INFO:tensorflow:SavedModel written to: /content/model/trained/export/exporter/temp-b'1556520069'/saved_model.pb


I0429 06:41:15.088508 140209631766400 builder_impl.py:414] SavedModel written to: /content/model/trained/export/exporter/temp-b'1556520069'/saved_model.pb


INFO:tensorflow:Loss for final step: 0.075892076.


I0429 06:41:15.225417 140209631766400 estimator.py:359] Loss for final step: 0.075892076.


({'acc': 0.979, 'loss': 0.06489962, 'global_step': 500}, [b'/content/model/trained/export/exporter/1556520069'])


In [31]:
predictions = model.predict(input_fn = pred_input_fn)
for items in predictions:
    print(items)

NameError: ignored

In [0]:
pred_path, eval_path